# Named Tensor

Now let's look at **Named Tensors**, which are a PyTorch feature that makes working with complex tensors (like medical images!) less error-prone and easier to read.

---

## Overview

### What is a Named Tensor?

Normally, tensors are indexed by **position only**:

In [1]:
import numpy
import torch

x = torch.randn(32, 1, 512, 512)  # (N, C, H, W)

If you want the **height dimension**, you need to remember it's at index `2`.
That's fine for simple 2D images — but with 5D CT/MRI volumes `(N, C, D, H, W)` or time-series, it's very easy to mix things up.

**Named Tensors** let you **attach names to dimensions**:

In [4]:
x = torch.randn(32, 1, 512, 512, names=('batch', 'channel', 'height', 'width'))

Now, instead of remembering that height is at position `2`, you can refer to it by name:

In [ ]:
print(x.names)

('batch', 'channel', 'height', 'width')


### Why is this Useful?

1. **Readability**
   You don't have to mentally track which index is which.

In [6]:
x.sum('height')   # instead of x.sum(dim=2)

tensor([[[ 33.5692, -45.9581, -14.5042,  ...,   1.4120,  -5.2512, -11.7449]],

        [[ -1.3578,  25.8050,  24.0318,  ...,   4.5506,  28.2631, -60.6962]],

        [[ 11.6369,   5.2120,   0.5251,  ...,  13.6742,   8.0234,  16.4690]],

        ...,

        [[-36.0494,   3.7133,  26.5676,  ...,  21.4684, -26.5332, -14.7450]],

        [[ 12.4113,   4.3388,  -2.9167,  ..., -23.2582,   3.0929,  42.1314]],

        [[  7.0003, -17.8118,   4.0587,  ...,  -7.0770,  22.7034,   4.0079]]],
       names=('batch', 'channel', 'width'))

2. **Safety (fewer bugs)**
   Operations align dimensions **by name**, not position:

In [7]:
a = torch.randn(10, 512, names=('batch', 'height'))
b = torch.randn(512, 20, names=('height', 'width'))

# Multiply across matching names automatically:
c = a @ b  # result has names ('batch', 'width')
print(c.names)

('batch', 'width')


3. **Complex medical tensors**
   In medical imaging, you often juggle `(N, C, D, H, W)` volumes. Named tensors prevent mistakes like swapping `D` and `H`.
   Example:

In [8]:
vol = torch.randn(4, 1, 128, 512, 512, names=('batch','channel','depth','height','width'))
proj = vol.mean('depth')  # MIP projection over slices
print(proj.names)

('batch', 'channel', 'height', 'width')


   Without names you'd write `vol.mean(dim=2)`, which is less clear.

4. **Self-documenting code**
   When someone else (or future-you) reads your code, names tell the story right away.



### 🔹 Limitation

* As of PyTorch 2.x, **Named Tensors are still experimental** and don't support *all* operations.
* You may need to `rename(None)` to go back to unnamed mode for some functions.




## Example: CT Volume Projection with Named Tensor

In [9]:
import torch

# 4 patients, 1 channel, 128 slices, 512x512 image
vol = torch.randn(4, 1, 128, 512, 512,
                  names=('batch','channel','depth','height','width'))

# Compute mean projection across depth
mip = vol.mean('depth')  # shape: (batch, channel, height, width)
print(mip.names)  # ('batch','channel','height','width')

('batch', 'channel', 'height', 'width')


Without names, you'd have to remember `.mean(dim=2)` (not `3` or `4`).

---

# 🔹 Summary

* **Named Tensors** = dimensions with labels.
* **Helps with**: readability, safety, reducing bugs, making code self-explanatory.
* **Especially valuable** in medical imaging, where tensors often have **5+ dimensions** `(N, C, D, H, W)` and mistakes are costly.
